# **Tensorflow를 이용한 MNIST Multi Layer Perceptron 실습하기**

In [46]:
import tensorflow.compat.v1 as tf
import numpy as np

tf.__version__

'2.4.1'

## **MLP Architecture**
input layer, hidden layer h1, h2, output layer logit. 10개의 output 노드가 10개의 softmax를 통해 prediction으로 변하고, 이 prediction을 y label과 비교를 함으로서 cross entropy를 가짐. 이 cross entropy를 최저로 가지는 것을 목적 함수로 해 adam optimizer를 돌려 architecture를 최적화 한다.
<p align="center"><img src="https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/simple_mlp_mnist.png" height="450"></p>

## **MNIST Data**

In [2]:
# mnist data load
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [3]:
print(x_train.shape)
print(y_train.shape)

(60000, 28, 28)
(60000,)


train data는 **60000**개의 sample을 가진다.  
test data는 **60000**개의 sample을 가진다.  
모든 data는 **28*28** pixels
  
  MNIST는 사람이 손으로 쓴 숫자로 이루어지는 대형 데이터베이스다. 아래 이미지는 28*28 pexels로 구성되며 흰색과 검정색으로 이루어진 gray scale image [0 to 255]다.

<p align="center"><img src="https://mblogthumb-phinf.pstatic.net/MjAxOTAxMTZfMTg4/MDAxNTQ3NjQ2MzI5OTQ0.257wNc_ErINd8lMDTaor5ls3UP0T2lD1yV_KoC_Ik6Mg.OGV-dRDH3m9U48AhlEaptGI9D8tHsvP-OZbPYyofin4g.PNG.garden2040/Figure_1.png?type=w800" height="300"></p>


## **Split Train data into Train and Validation data**
training 중 validation은 다음과 같은 장점을 가진다.  
1. training 중 validation score를 이용해 hyper parameter를 수정할 수 있다.
2. training score가 올라가는 동안 validation score(accuracy)가 변화가 없을 시 **early stopping**을 제공한다. (**overfitting** 방지)


train data 60000개 중 10000개의 데이터를 validation data로 사용한다.

In [4]:
x_val = x_train[50000:60000]
x_train = x_train[0:50000]
y_val = y_train[50000:60000]
y_train = y_train[0:50000]

## **Reshape**

모든 pixels을 hidden layer에 fully connected (완전 연결)하려면 (28,28)을 (28x28, 1)로 reshape 해야한다.  
== row * col shape를 28x28개의 item이 있는 행렬로 flatten 해야 한다.

<p align="center"><img src="https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/reshape_mnist.png" height="420"></p>

In [5]:
x_train = x_train.reshape(50000, 784)
x_val = x_val.reshape(10000, 784)
x_test = x_test.reshape(10000, 784)

print(x_train.shape)
print(x_test.shape)

(50000, 784)
(10000, 784)


In [6]:
x_train[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18,
       126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 17

## **Normalize data**
정규화는 모든 입력 feature를 동일한 범위로 만들어 분산을 줄임으로써 학습 속도를 향상시키거나, 더 나은 성능을 돕는다.  
MNIST 데이터 셋은 [0 to 255]의 값을 가지므로 정규화를 할 시 variance만 줄일 수 있다.  
  
MNIST는 MLP architecture로 표준화하는 것보다 정규화하는 것이 더 낫다. ReLU가 peed forward와 back propagation에서 모두 0을 다르게 다루기 때문. MNIST에서 [1 to 255]는 손으로 쓴 것을 의미하므로 아무 것도 쓰지 않은 0을 다르게 취급하는 것이 중요하다.


In [7]:
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')

# 255로 나누게 되면 [0 to 1]의 값을 가지게 됨
gray_scale = 255 
x_train /= gray_scale
x_val /= gray_scale
x_test /= gray_scale

## **Label to One Hot Encoding Value**


In [8]:
# class를 10으로 해 y값들을 모두 One Hot Encoding한다. (1~10까지의 숫자를 0~9번 노드로)
num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [9]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

## **Tensorflow MLP Graph**

<p align="center"><img src="https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/simple_mlp_mnist.png" height="450"></p>

In [83]:
# tensorflow 2.x에서는 session을 정의하고 run을 수행하는 과정이 생략되고 바로 실행되는 형태가 되었으므로 아래와 같은 옵션을 추가해준다.
tf.compat.v1.disable_eager_execution()
# input으로 784개
x = tf.compat.v1.placeholder(tf.float32, [None, 784])
# [None, 10] : 총 몇 개의 sample이 들어올지 모르겠지만 10개의 숫자를 구분할 것이다.
y = tf.compat.v1.placeholder(tf.float32, [None, 10])

In [84]:
def mlp(x):
  # hidden layer1
  w1 = tf.Variable(tf.compat.v1.random_uniform([784, 256])) # 784개의 input을 받고, 256개의 node를 만든다.
  b1 = tf.Variable(tf.zeros([256]))
  h1 = tf.nn.relu(tf.matmul(x, w1) + b1) # activation func : ReLU
  # hidden layer2
  w2 = tf.Variable(tf.compat.v1.random_uniform([256, 128])) # 256개의 input을 받고, 128개의 node를 만든다.
  b2 = tf.Variable(tf.zeros([128]))
  h2 = tf.nn.relu(tf.matmul(h1, w2) + b2) # activation func : ReLU
  # output layer
  w3 = tf.Variable(tf.compat.v1.random_uniform([128, 10])) # 128개의 input을 받고, 10개의 node를 만든다.
  b3 = tf.Variable(tf.zeros([10]))
  logits = tf.matmul(h2, w3) + b3

  return logits

cross entropy를 하기 위해 softmax를 많이 사용한다. 이 logic value를 softmax에 넣어서 prediction을 받는다. 이 cross entropy를 minimize하는 것이 adam optimizer.

In [85]:
logits = mlp(x)

In [86]:
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = y))

In [87]:
train_op = tf.compat.v1.train.AdamOptimizer(learning_rate=0.01).minimize(loss_op)

In [88]:
# initialize
init = tf.global_variables_initializer()

# train hyperparameters
epoch_cnt = 30
batch_size = 1000 # 1000개씩 넣으면서 architecture 최적화
iteration = len(x_train) // batch_size # input이 50000개니까 총 50번의 iteration

# Start training
with tf.Session() as sess:
    # Run the initializer
    sess.run(init)
    for epoch in range(epoch_cnt): # epoch 돌아감
        avg_loss = 0.
        start = 0; end = batch_size
        
        for i in range(iteration): # batch 돌아감
            _, loss = sess.run([train_op, loss_op], feed_dict={x: x_train[start: end], y: y_train[start: end]})
            start += batch_size; end += batch_size
            # Compute average loss
            avg_loss += loss / iteration
            
        # Validate model
        preds = tf.nn.softmax(logits)  # Apply softmax to logits
        correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(y, 1)) # 몇개가 맞았는지
        # Calculate accuracy
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        cur_val_acc = accuracy.eval({x: x_val, y: y_val})
        print("epoch: "+str(epoch)+", validation accuracy: " + str(cur_val_acc) +', loss: '+str(avg_loss))
    
    # Test model
    preds = tf.nn.softmax(logits)  # Apply softmax to logits
    correct_prediction = tf.equal(tf.argmax(preds, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("[Test Accuracy] :", accuracy.eval({x: x_test, y: y_test}))

epoch: 0, validation accuracy: 0.1923, loss: 9561.096894531249
epoch: 1, validation accuracy: 0.7839, loss: 283.1611603164674
epoch: 2, validation accuracy: 0.8761, loss: 13.110803556442264
epoch: 3, validation accuracy: 0.8816, loss: 8.532276582717895
epoch: 4, validation accuracy: 0.8855, loss: 6.908381538391112
epoch: 5, validation accuracy: 0.889, loss: 5.965815544128417
epoch: 6, validation accuracy: 0.8937, loss: 5.653503303527832
epoch: 7, validation accuracy: 0.8959, loss: 5.621636781692505
epoch: 8, validation accuracy: 0.9058, loss: 4.810211319923401
epoch: 9, validation accuracy: 0.9142, loss: 5.174941515922548
epoch: 10, validation accuracy: 0.9125, loss: 3.8908423376083365
epoch: 11, validation accuracy: 0.8658, loss: 4.601394052505493
epoch: 12, validation accuracy: 0.877, loss: 4.431019220352173
epoch: 13, validation accuracy: 0.9043, loss: 4.767708916664124
epoch: 14, validation accuracy: 0.6411, loss: 33.19531376838684
epoch: 15, validation accuracy: 0.9119, loss: 15.1